In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import numpy as np
import pandas as pd
import torch

from tsdm.encoders.numerical import (
    BoundaryEncoder,
    MinMaxScaler,
    get_broadcast,
    invert_axes,
)

In [ ]:
shape = (3, 2, 1, 2)
axis = None

arr = np.random.randn(*shape)
broadcast = get_broadcast(arr.shape, axis=axis)
m = np.mean(arr, axis=axis)
m_ref = np.mean(arr, axis=axis, keepdims=True)
assert m[broadcast].shape == m_ref.shape

In [ ]:
X = np.random.randn(10)
encoder = MinMaxScaler(axis=None)
encoder.fit(X)
encoded = encoder.encode(X)
decoded = encoder.decode(encoded)

In [ ]:
invert_axes(3, ())

In [ ]:
2 not in ()

In [ ]:
encoder.xmin

In [ ]:
X = np.random.randn(100, 5)
encoder = MinMaxScaler(axis=-1)
encoder.fit(X)
encoded = encoder.encode(X)
decoded = encoder.decode(encoded)

In [ ]:
np.std([1.1])

In [ ]:
x = {1, 2}

In [ ]:
match {2,1}:
    case {1,2}:
        print("sets are unordered")
    case (2, 1):
        print("sad trombone")

In [ ]:
match {2: 0, 1: 0}:
    case {1: 0, 2: 0}:
        print("Sets are unordered!")

In [ ]:
pd.DataFrame.where(s.to_frame(), m, 0)

In [ ]:
class Foo:
    @property
    def bar(self):
        print("computing")
        return 42

In [ ]:
class Color(enum.Enum):
    foo = 123
    bar = 1.3

In [ ]:
Color.bar

In [ ]:
s = pd.Series([1, 2, 3])
m = s > 2

In [ ]:
x = lambda s=0: s

In [ ]:
torch.tensor([1.3, 0.0]).to(bool)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(encoded.T[3]);

In [ ]:
X = np.random.randn(10)
encoder = MinMaxScaler()
encoder.fit(X)
encoded = encoder.encode(X)
decoded = encoder.decode(encoded)

In [ ]:
encoder

In [ ]:
get_broadcast(X.shape, axis=encoder.axis)

In [ ]:
get_broadcast((10, 5), axis=())

In [ ]:
arr.shape

In [ ]:
np.mean(arr, axis=(-1, -2)).shape

In [ ]:
def repr_broadcast(x):
    items = (":" if y == slice(None) else str(y) for y in x)
    return f"[{', '.join(items)}]"

In [ ]:
for shape in ((3, 4), (7,)):
    for axis in ((1, -1), (-1,), 0, None, ()):
        if isinstance(axis, tuple) and len(axis) > len(shape):
            continue

        broadcast = get_broadcast(shape, axis=axis)
        print(f"{str(shape):16s}{str(axis):16s}{repr_broadcast(broadcast)}")

In [ ]:
for shape in ((3, 4), (7,)):
    for axis in ((1, -1), (-1,), 0, None, ()):
        if isinstance(axis, tuple) and len(axis) > len(shape):
            continue

        broadcast = get_broadcast(shape, axis=axis, keep_axis=True)
        print(f"{str(shape):16s}{str(axis):16s}{repr_broadcast(broadcast)}")

In [ ]:
get_broadcast(X.shape, axis=encoder.axis, keep_axis=False)

In [ ]:
X = np.random.randn(10, 5)
encoder = MinMaxScaler(axis=-1)
encoder.fit(X)

In [ ]:
encoded = encoder.encode(X)
decoded = encoder.decode(encoded)

In [ ]:
get_broadcast(X.shape, axis=encoder.axis, keep_axis=True)

In [ ]:
get_broadcast((10, 5), axis=0)

In [ ]:
get_broadcast((10, 5), axis=-1)

In [ ]:
get_broadcast((7,), axis=None)

In [ ]:
encoder.encode(X)

In [ ]:
get_broadcast(X.shape, axis=encoder.axis)

In [ ]:
encoder